In [90]:
import pandas as pd
import numpy as np

In [11]:
path_blood= r"Documents/ESTÁGIO LGHM - BIOINFO - CAMILA/DATASETS/TCGA_mutations_stomach_blood.json"
path_tissue= r"Documents/ESTÁGIO LGHM - BIOINFO - CAMILA/DATASETS/TCGA_mutations_stomach_solid_tissue.json"
path_tumor= r"Documents/ESTÁGIO LGHM - BIOINFO - CAMILA/DATASETS/TCGA_mutations_stomach_primary_tumor.json"

In [247]:
blood = pd.read_json(path_blood, orient='records')
tissue = pd.read_json(path_tissue, orient='records')
primary_tumor= pd.read_json(path_tumor, orient='records')

In [248]:
def gene(json):
    lista = []
    for i in range(len(json)):
        lista.append(json['consequence'][i][0]['transcript']['gene']['symbol'])
    indices = [idx for idx, gene in enumerate(lista) if gene =='CFTR']
    return indices

In [249]:
idx_blood = gene(blood)
idx_tumor = gene(primary_tumor)
idx_tissue = gene(tissue)

In [250]:
idx_blood

[13247,
 21617,
 29333,
 29416,
 31351,
 44415,
 63536,
 64965,
 82275,
 91483,
 94223,
 96214,
 102981,
 103672,
 111835,
 119743,
 121146]

In [258]:
def annotation(tipo, idx, name):
    lista = [[] for x in range(6)]
    for ix in idx:
        data = tipo['consequence'][ix]
        for i in range(len(data)):
            t = data[i]['transcript']
            poly_imp = t['annotation']['polyphen_impact']
            sift_imp = t['annotation']['sift_impact']
            vep_imp = t['annotation']['vep_impact']
            conseq_type = t['consequence_type']
            gene_symbol = t['gene']['symbol']
            lista[0].append(poly_imp)
            lista[1].append(sift_imp)
            lista[2].append(vep_imp)
            lista[3].append(conseq_type)
            lista[4].append(gene_symbol)
            lista[5].append(tipo['genomic_dna_change'][ix])
    lista = np.reshape(lista,(6,-1)).T
    df = pd.DataFrame(lista, index = [name for s in range(len(lista))],
             columns = ['polyphen_imp', 'sift_impact', 'vep_impact', 'consequence_type','gene_symbol','genomic_dna_change'])
    df.drop_duplicates(inplace=True)
    return df

In [261]:
df_tissue = annotation(tissue, idx_tissue, 'solid tissue normal')
df_blood = annotation(blood, idx_blood, 'blood derived normal')
df_tumor = annotation(primary_tumor, idx_tumor, 'primary tumor')

In [262]:
df_stomach = pd.concat([df_tissue, df_blood, df_tumor])

In [268]:
df_stomach.to_csv('TCGA_STOMACH_CFTR.csv')

In [267]:
type(df_stomach)

pandas.core.frame.DataFrame